## Importing Libraries

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
import re
import string

## Importing Dataset

In [5]:
df = pd.read_csv("news.csv")

In [6]:
df.head(5)

title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept transgender recruits o...   
2  Senior U.S. Republican senator: 'Let Mr. Muell...   
3  FBI Russia probe helped by Australian diplomat...   
4  Trump wants Postal Service to charge 'much mor...   

                                                text       subject  \
0  WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1  WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2  WASHINGTON (Reuters) - The special counsel inv...  politicsNews   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  politicsNews   
4  SEATTLE/WASHINGTON (Reuters) - President Donal...  politicsNews   

                 date  isReal  
0  December 31, 2017        1  
1  December 29, 2017        1  
2  December 31, 2017        1  
3  December 30, 2017        1  
4  December 29, 2017        1

In [7]:
df_true=df[df['isReal']==1]
df_fake=df[df['isReal']==0]

## Inserting a column "class" as target feature

In [8]:
df_fake["class"] = 0
df_true["class"] = 1

C:\Users\Mostafa\AppData\Local\Temp\ipykernel_5088\3873677238.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fake["class"] = 0
C:\Users\Mostafa\AppData\Local\Temp\ipykernel_5088\3873677238.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_true["class"] = 1


In [9]:
df_fake.shape, df_true.shape

((23481, 6), (21417, 6))

In [10]:
# Removing last 10 rows for manual testing
df_fake_manual_testing = df_fake.tail(10)
for i in range(23480,23470,-1):
    df_fake.drop([i], axis = 0, inplace = True)
    
    
df_true_manual_testing = df_true.tail(10)
for i in range(21416,21406,-1):
    df_true.drop([i], axis = 0, inplace = True)

C:\Users\Mostafa\AppData\Local\Temp\ipykernel_5088\2329372083.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fake.drop([i], axis = 0, inplace = True)
C:\Users\Mostafa\AppData\Local\Temp\ipykernel_5088\2329372083.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fake.drop([i], axis = 0, inplace = True)
C:\Users\Mostafa\AppData\Local\Temp\ipykernel_5088\2329372083.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fake.d

In [11]:
df_fake.shape, df_true.shape

((23471, 6), (21407, 6))

In [12]:
df_fake_manual_testing["class"] = 0
df_true_manual_testing["class"] = 1

C:\Users\Mostafa\AppData\Local\Temp\ipykernel_5088\860779283.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_fake_manual_testing["class"] = 0
C:\Users\Mostafa\AppData\Local\Temp\ipykernel_5088\860779283.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_true_manual_testing["class"] = 1


In [13]:
df_fake_manual_testing.head(10)

title  \
44888  Seven Iranians freed in the prisoner swap have...   
44889                      #Hashtag Hell & The Fake Left   
44890  Astroturfing: Journalist Reveals Brainwashing ...   
44891          The New American Century: An Era of Fraud   
44892  Hillary Clinton: ‘Israel First’ (and no peace ...   
44893  McPain: John McCain Furious That Iran Treated ...   
44894  JUSTICE? Yahoo Settles E-mail Privacy Class-ac...   
44895  Sunnistan: US and Allied ‘Safe Zone’ Plan to T...   
44896  How to Blow $700 Million: Al Jazeera America F...   
44897  10 U.S. Navy Sailors Held by Iranian Military ...   

                                                    text      subject  \
44888  21st Century Wire says This week, the historic...  Middle-east   
44889   By Dady Chery and Gilbert MercierAll writers ...  Middle-east   
44890  Vic Bishop Waking TimesOur reality is carefull...  Middle-east   
44891  Paul Craig RobertsIn the last years of the 20t...  Middle-east   
44892  Robert Fantina CounterpunchAlthough the United...  Middle-east   
44893  21st Century Wire says As 21WIRE reported earl...  Middle-east   
44894  21st Century Wire says It s a familiar theme. ...  Middle-east   
44895  Patrick Henningsen  21st Century WireRemember ...  Middle-east   
44896  21st Century Wire says Al Jazeera America will...  Middle-east   
44897  21st Century Wire says As 21WIRE predicted in ...  Middle-east   

                   date  isReal  class  
44888  January 20, 2016       0      0  
44889  January 19, 2016       0      0  
44890  January 19, 2016       0      0  
44891  January 19, 2016       0      0  
44892  January 18, 2016       0      0  
44893  January 16, 2016       0      0  
44894  January 16, 2016       0      0  
44895  January 15, 2016       0      0  
44896  January 14, 2016       0      0  
44897  January 12, 2016       0      0

In [14]:
df_true_manual_testing.head(10)

title  \
21407  Mata Pires, owner of embattled Brazil builder ...   
21408  U.S., North Korea clash at U.N. forum over nuc...   
21409  U.S., North Korea clash at U.N. arms forum on ...   
21410  Headless torso could belong to submarine journ...   
21411  North Korea shipments to Syria chemical arms a...   
21412  'Fully committed' NATO backs new U.S. approach...   
21413  LexisNexis withdrew two products from Chinese ...   
21414  Minsk cultural hub becomes haven from authorities   
21415  Vatican upbeat on possibility of Pope Francis ...   
21416  Indonesia to buy $1.14 billion worth of Russia...   

                                                    text    subject  \
21407  SAO PAULO (Reuters) - Cesar Mata Pires, the ow...  worldnews   
21408  GENEVA (Reuters) - North Korea and the United ...  worldnews   
21409  GENEVA (Reuters) - North Korea and the United ...  worldnews   
21410  COPENHAGEN (Reuters) - Danish police said on T...  worldnews   
21411  UNITED NATIONS (Reuters) - Two North Korean sh...  worldnews   
21412  BRUSSELS (Reuters) - NATO allies on Tuesday we...  worldnews   
21413  LONDON (Reuters) - LexisNexis, a provider of l...  worldnews   
21414  MINSK (Reuters) - In the shadow of disused Sov...  worldnews   
21415  MOSCOW (Reuters) - Vatican Secretary of State ...  worldnews   
21416  JAKARTA (Reuters) - Indonesia will buy 11 Sukh...  worldnews   

                   date  isReal  class  
21407  August 22, 2017        1      1  
21408  August 22, 2017        1      1  
21409  August 22, 2017        1      1  
21410  August 22, 2017        1      1  
21411  August 21, 2017        1      1  
21412  August 22, 2017        1      1  
21413  August 22, 2017        1      1  
21414  August 22, 2017        1      1  
21415  August 22, 2017        1      1  
21416  August 22, 2017        1      1

In [15]:
df_manual_testing = pd.concat([df_fake_manual_testing,df_true_manual_testing], axis = 0)
df_manual_testing.to_csv("manual_testing.csv")

## Merging True and Fake Dataframes

In [16]:
df_merge = pd.concat([df_fake, df_true], axis =0 )
df_merge.head(10)

title  \
21417   Donald Trump Sends Out Embarrassing New Year’...   
21418   Drunk Bragging Trump Staffer Started Russian ...   
21419   Sheriff David Clarke Becomes An Internet Joke...   
21420   Trump Is So Obsessed He Even Has Obama’s Name...   
21421   Pope Francis Just Called Out Donald Trump Dur...   
21422   Racist Alabama Cops Brutalize Black Boy While...   
21423   Fresh Off The Golf Course, Trump Lashes Out A...   
21424   Trump Said Some INSANELY Racist Stuff Inside ...   
21425   Former CIA Director Slams Trump Over UN Bully...   
21426   WATCH: Brand-New Pro-Trump Ad Features So Muc...   

                                                    text subject  \
21417  Donald Trump just couldn t wish all Americans ...    News   
21418  House Intelligence Committee Chairman Devin Nu...    News   
21419  On Friday, it was revealed that former Milwauk...    News   
21420  On Christmas day, Donald Trump announced that ...    News   
21421  Pope Francis used his annual Christmas Day mes...    News   
21422  The number of cases of cops brutalizing and ki...    News   
21423  Donald Trump spent a good portion of his day a...    News   
21424  In the wake of yet another court decision that...    News   
21425  Many people have raised the alarm regarding th...    News   
21426  Just when you might have thought we d get a br...    News   

                    date  isReal  class  
21417  December 31, 2017       0      0  
21418  December 31, 2017       0      0  
21419  December 30, 2017       0      0  
21420  December 29, 2017       0      0  
21421  December 25, 2017       0      0  
21422  December 25, 2017       0      0  
21423  December 23, 2017       0      0  
21424  December 23, 2017       0      0  
21425  December 22, 2017       0      0  
21426  December 21, 2017       0      0

In [17]:
df_merge.columns

Index(['title', 'text', 'subject', 'date', 'isReal', 'class'], dtype='object')

## Removing columns which are not required

In [18]:
df = df_merge.drop(["title", "subject","date"], axis = 1)

In [19]:
df.isnull().sum()

text      0
isReal    0
class     0
dtype: int64

## Random Shuffling the dataframe

In [20]:
df = df.sample(frac = 1)

In [21]:
df.head()

text  isReal  class
1003   HAVANA (Reuters) - The United States said on T...       1      1
7483   BRUSSELS (Reuters) - EU foreign policy chief F...       1      1
28059  Far too many people don t understand the aweso...       0      0
39987  There is no question that Whoopi Goldberg is a...       0      0
5758   WASHINGTON (Reuters) - Rex Tillerson’s job as ...       1      1

In [22]:
df.reset_index(inplace = True)
df.drop(["index"], axis = 1, inplace = True)

In [23]:
df.columns

Index(['text', 'isReal', 'class'], dtype='object')

In [24]:
df.head()

text  isReal  class
0  HAVANA (Reuters) - The United States said on T...       1      1
1  BRUSSELS (Reuters) - EU foreign policy chief F...       1      1
2  Far too many people don t understand the aweso...       0      0
3  There is no question that Whoopi Goldberg is a...       0      0
4  WASHINGTON (Reuters) - Rex Tillerson’s job as ...       1      1

## Creating a function to process the texts

In [25]:
def wordopt(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) 
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)    
    return text

<>:3: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:9: SyntaxWarning: invalid escape sequence '\w'
<>:3: SyntaxWarning: invalid escape sequence '\['
<>:5: SyntaxWarning: invalid escape sequence '\S'
<>:9: SyntaxWarning: invalid escape sequence '\w'
C:\Users\Mostafa\AppData\Local\Temp\ipykernel_5088\3432647204.py:3: SyntaxWarning: invalid escape sequence '\['
  text = re.sub('\[.*?\]', '', text)
C:\Users\Mostafa\AppData\Local\Temp\ipykernel_5088\3432647204.py:5: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub('https?://\S+|www\.\S+', '', text)
C:\Users\Mostafa\AppData\Local\Temp\ipykernel_5088\3432647204.py:9: SyntaxWarning: invalid escape sequence '\w'
  text = re.sub('\w*\d\w*', '', text)


In [26]:
df["text"] = df["text"].apply(wordopt)

## Defining dependent and independent variables

In [27]:
x = df["text"]
y = df["class"]

## Splitting Training and Testing

In [28]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

## Convert text to vectors

In [29]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_df=0.7)
xv_train = vectorizer.fit_transform(x_train)
xv_test = vectorizer.transform(x_test)


## Logistic Regression

In [30]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(xv_train,y_train)

LogisticRegression()

In [31]:
pred_lr=LR.predict(xv_test)

In [32]:
LR.score(xv_test, y_test)

0.9854055258467023

In [33]:
print(classification_report(y_test, pred_lr))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4681
           1       0.98      0.99      0.98      4295

    accuracy                           0.99      8976
   macro avg       0.99      0.99      0.99      8976
weighted avg       0.99      0.99      0.99      8976



## Decision Tree Classification

In [34]:
from sklearn.tree import DecisionTreeClassifier

DT = DecisionTreeClassifier()
DT.fit(xv_train, y_train)

DecisionTreeClassifier()

In [35]:
pred_dt = DT.predict(xv_test)

In [36]:
DT.score(xv_test, y_test)

0.9964349376114082

In [37]:
print(classification_report(y_test, pred_dt))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4681
           1       1.00      1.00      1.00      4295

    accuracy                           1.00      8976
   macro avg       1.00      1.00      1.00      8976
weighted avg       1.00      1.00      1.00      8976



## Gradient Boosting Classifier

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

GBC = GradientBoostingClassifier(random_state=0)
GBC.fit(xv_train, y_train)

In [ ]:
pred_gbc = GBC.predict(xv_test)

In [ ]:
GBC.score(xv_test, y_test)

0.9962121212121212

In [ ]:
print(classification_report(y_test, pred_gbc))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      4726
           1       0.99      1.00      1.00      4250

    accuracy                           1.00      8976
   macro avg       1.00      1.00      1.00      8976
weighted avg       1.00      1.00      1.00      8976



## Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

RFC = RandomForestClassifier(random_state=0)
RFC.fit(xv_train, y_train)

RandomForestClassifier(random_state=0)

In [ ]:
pred_rfc = RFC.predict(xv_test)

In [ ]:
RFC.score(xv_test, y_test)

0.990530303030303

In [ ]:
print(classification_report(y_test, pred_rfc))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      4726
           1       0.99      0.99      0.99      4250

    accuracy                           0.99      8976
   macro avg       0.99      0.99      0.99      8976
weighted avg       0.99      0.99      0.99      8976



## Model Testing

In [ ]:
def output_lable(n):
    if n == 0:
        return "Fake News"
    elif n == 1:
        return "Not A Fake News"
    
def manual_testing(news):
    testing_news = {"text":[news]}
    new_def_test = pd.DataFrame(testing_news)
    new_def_test["text"] = new_def_test["text"].apply(wordopt) 
    new_x_test = new_def_test["text"]
    new_xv_test = vectorization.transform(new_x_test)
    pred_LR = LR.predict(new_xv_test)
    pred_DT = DT.predict(new_xv_test)
    pred_GBC = GBC.predict(new_xv_test)
    pred_RFC = RFC.predict(new_xv_test)

    return print("\n\nLR Prediction: {} \nDT Prediction: {} \nGBC Prediction: {} \nRFC Prediction: {}".format(output_lable(pred_LR[0]),                                                                                                       output_lable(pred_DT[0]), 
                                                                                                              output_lable(pred_GBC[0]), 
                                                                                                              output_lable(pred_RFC[0])))

In [ ]:
news = str(input())
manual_testing(news)



LR Prediction: Fake News 
DT Prediction: Fake News 
GBC Prediction: Fake News 
RFC Prediction: Fake News


In [ ]:
news = str(input())
manual_testing(news)



LR Prediction: Not A Fake News 
DT Prediction: Fake News 
GBC Prediction: Fake News 
RFC Prediction: Not A Fake News


In [ ]:
news = str(input())
manual_testing(news)



LR Prediction: Not A Fake News 
DT Prediction: Fake News 
GBC Prediction: Fake News 
RFC Prediction: Not A Fake News


# Save Models

In [ ]:
from joblib import dump, load

dump(vectorization, 'models/vectorizer.pkl')
dump(LR, 'models/LR_model.joblib')
dump(DT, 'models/DT_model.joblib')
dump(GBC, 'models/GBC_model.joblib')
dump(RFC, 'models/RFC_model.joblib')

NameError: name 'vectorization' is not defined